<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install getpass
%pip install pypdf

In [9]:
import openai
from openai import OpenAI
import os

client = OpenAI()

In [12]:
training_json = []

In [28]:
def store_json(chunk, targ):
    json_obj = { "instruction" : "", "input" : "", "output" : chunk }
    targ.append(json_obj)

In [33]:
def chunk_gpt(text, targ):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "},
            {"role": "user", "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"}
        ]
    )
    store_json(response.choices[0].message.content.strip().split('\n'), targ)

In [27]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)

print(len(filenames))
print(filenames)

45
['Alan W. Watts_ Joan Watts_ Anne Watts - -New World Library (2018)_djvu.txt', 'Alan Watts in the academy, essays and lecture (2017)_djvu.txt', 'Alan Watts_ Mark Watts - Talking Zen_ Reflections on Mind, Myth, and the Magic of Life-Shambhala (2022)_djvu.txt', 'Alan-Watts-Mark-Watts-Talking-Zen_djvu.txt', 'Columbus, Peter (ed.) - Alan Watts_ In the Academy (SUNY, 2017)_djvu.txt', 'Snelling, John (ed.) - Early Writings of Alan Watts (Celestial Arts, 1987)_djvu.txt', 'Watts, Alan - Become What You Are (Shambhala, 2014)_djvu.txt', 'Watts, Alan - Behold the Spirit (Vintage, 2011)_djvu.txt', 'Watts, Alan - Beyond Theology (Vintage, 1973)_djvu.txt', 'Watts, Alan - Book, The (Vintage, 1989)_djvu.txt', 'Watts, Alan - Buddhism_ The Religion of No-Religion (Tuttle, 1999)_djvu.txt', 'Watts, Alan - Cloud-Hidden, Whereabouts Unknown (Vintage, 1974)_djvu.txt', 'Watts, Alan - Does It Matter (New World Library, 2007)_djvu.txt', 'Watts, Alan - Eastern Wisdom (MJF, 2000)_djvu.txt', 'Watts, Alan - East

In [32]:
chunk_size = 5000
text_chunks = []

for i in range (1,2):
    
    with open(f'books/{filenames[i]}', 'r', encoding='utf-8', errors='replace') as file:
        content = file.read().replace('\n','')

        text_chunks += [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        print(f'chunks: {len(text_chunks)}')
        print(text_chunks[0])

chunks: 197
Alan Watis- CSSdy5 and 9 ¥ - . Edited and with Br introduction by j Peter J. Columbus and Donadrian L. Rice 4 L SEK VE, HB Vig Coe ie SUNY SERIES IN TRANSPERSONAL AND HUMANISTIC PSYCHOLOGY Richard D. Mann, editor SE SO, MWe OL Otebio essays and lectures Edited and with an introduction by Peter J. Columbus and Donadrian L. Rice On the cover: Portrait of Alan Watts by Fred Richards—Daishi, from the artist’s collection. Published by STATE UNIVERSITY OF NEW YORK PRESS Albany © 2017 State University of New York All rights reserved Printed in the United States of America No part of this book may be used or reproduced in any manner whatsoever without written permission. No part of this book may be stored in a retrieval system or transmitted in any form or by any means including electronic, electrostatic, magnetic tape, mechanical, photocopying, recording, or otherwise without the prior permission in writing of the publisher. For information, contact State University of New York Pr

In [34]:
processed_json = []

for chunk in text_chunks:
    gpt_output = chunk_gpt(chunk, processed_json)